In [1]:
import os
import tempfile
import matplotlib.pyplot as plt
import glob
from pymongo import MongoClient, InsertOne
import pandas as pd
import numpy as np
from functools import reduce
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

database = "data_8"
col = "items"

2023-05-08 17:25:25.640098: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
mongo = MongoClient()
df_meta = pd.DataFrame(list(mongo[database][col].find()))
df_meta['jobTitle'] = df_meta.categoricalProps.apply(lambda x: x["jobTitle"][0])
df_meta['skills'] = df_meta.categoricalProps.apply(lambda x: ','.join(x["skills"]))
df_meta['industries'] = df_meta.categoricalProps.apply(lambda x: ','.join(x["industries"]))
df_meta['locations'] = df_meta.categoricalProps.apply(lambda x: ','.join(x["locations"]))
df_meta['jobLevel'] = df_meta.categoricalProps.apply(lambda x: ','.join(x["jobLevel"]))
df_meta['availableDate'] = df_meta.dateProps.apply(lambda x: x["availableDate"])

In [3]:
df = pd.read_parquet(glob.glob("/home/spark/ylv/data/navidata/*.parquet"))
ratings = df.merge(df_meta, left_on="targetEntityId", right_on="_id", how="left")
ratings = ratings.dropna()

In [4]:
@tf.keras.utils.register_keras_serializable()
def tokenization(t):
    return tf.strings.split(t, ',')

In [18]:
user_model_3 = tf.keras.models.load_model('/home/spark/ylv/workplace/navi_user_model_3')
item_model_3 = tf.keras.models.load_model('/home/spark/ylv/workplace/navi_item_model_3')
user_model_2 = tf.keras.models.load_model('/home/spark/ylv/workplace/rec_online_training/navi_user_model_2')
item_model_2 = tf.keras.models.load_model('/home/spark/ylv/workplace/rec_online_training/navi_item_model_2')
user_model_1 = tf.keras.models.load_model('/home/spark/ylv/workplace/rec_online_training/navi_user_model_1')
item_model_1 = tf.keras.models.load_model('/home/spark/ylv/workplace/rec_online_training/navi_item_model_1')

In [19]:
movies = tf.data.Dataset.from_tensor_slices({
    "job_id": df_meta._id.values.tolist(),
    "category": df_meta.industries.values.tolist(),
    "location": df_meta.locations.values.tolist(),
    "level": df_meta.jobLevel.values.tolist(),
    "job_title": df_meta.jobTitle.values.tolist(),
    "skill_text": df_meta.skills.values.tolist()
})

index1 = tfrs.layers.factorized_top_k.BruteForce(user_model_1, k=10)
index1.index_from_dataset(
        movies.batch(500).map(
            lambda x: (x["job_id"], item_model_1(x))
        )
)

index2 = tfrs.layers.factorized_top_k.BruteForce(user_model_2, k=10)
index2.index_from_dataset(
        movies.batch(500).map(
            lambda x: (x["job_id"], item_model_2(x))
        )
)

index3 = tfrs.layers.factorized_top_k.BruteForce(user_model_3, k=10)
index3.index_from_dataset(
        movies.batch(500).map(
            lambda x: (x["job_id"], item_model_3(x))
        )
)

In [25]:
vocab_base_1 = user_model_1.get_layer("user_embedding_model").get_layer("user_stringlookup_layer").get_vocabulary()
vocab_base_2 = user_model_2.get_layer("user_embedding_model").get_layer("user_stringlookup_layer").get_vocabulary()
vocab_base_3 = user_model_3.get_layer("user_embedding_model").get_layer("user_stringlookup_layer").get_vocabulary()
vocab_base = list(set(vocab_base_1).intersection(set(vocab_base_2)).intersection(set(vocab_base_3)))

In [212]:
u = np.random.choice(vocab_base)
# u = "171.240.26.108, 10.122.10.10"
print("========================HISTORY==============================")
ratings[ratings.entityId == u][["targetEntityId", "jobTitle", "industries", "jobLevel", "locations", "skills"]]
# score, pred = index({"user_id": np.array(["5749316"])})

========================HISTORY==============================


,targetEntityId,jobTitle,industries,jobLevel,locations,skills
2053361,1624162,nhan vien ke toan thanh toan,"Accounting,Auditing,Finance/Investment",Experienced (non-manager),Ha Noi,"accounting,financial report,ke toan thanh toan..."
2055099,1624162,nhan vien ke toan thanh toan,"Accounting,Auditing,Finance/Investment",Experienced (non-manager),Ha Noi,"accounting,financial report,ke toan thanh toan..."
2055148,1624162,nhan vien ke toan thanh toan,"Accounting,Auditing,Finance/Investment",Experienced (non-manager),Ha Noi,"accounting,financial report,ke toan thanh toan..."


In [213]:
# u = '183.80.135.240, 10.122.10.10'
score_3, pred_3 = index3({"user_id": np.array([u])})
pre_items_3 = list(map(lambda x: x.decode('ascii'), pred_3.numpy()[0]))
check_df_3 = df_meta[df_meta._id.isin(list(map(lambda x: x.decode('ascii'), pred_3.numpy()[0])))][["_id","jobTitle", "industries", "locations", "jobLevel", "skills"]]
check_df_3["priority"] = check_df_3._id.apply(lambda x: pre_items_3.index(x))
check_df_3.sort_values(by=["priority"], ascending=True)

,_id,jobTitle,industries,locations,jobLevel,skills,priority
3420,1624162,nhan vien ke toan thanh toan,"Accounting,Auditing,Finance/Investment",Ha Noi,Experienced (non-manager),"accounting,financial report,ke toan thanh toan...",0
3882,1624706,nhan vien dich vu ke toan han hcm,"Accounting,Auditing,Finance/Investment","Ho Chi Minh,Ha Noi",Experienced (non-manager),"ke toan tong hop,kiem toan,auditing,tax consul...",1
3481,1624244,nhan vien ke toan,"Accounting,Auditing,Electrical/Electronics",Ha Noi,Experienced (non-manager),"hach toan,phan mem ke toan,cong no chung tu",2
3363,1624093,nhan vien ke toan,"Accounting,Auditing,Finance/Investment",Ha Noi,Experienced (non-manager),"kiem toan,ke toan tong hop,ke toan,accounting,...",3
1578,1621753,ke toan tong hop,"Accounting,Auditing,Finance/Investment",Ha Noi,Experienced (non-manager),"tai chinh,kiem toan,ke toan tong hop,general a...",4
2910,1623490,it support chap nhan sinh vien moi ra truong,"IT - Software,IT - Hardware/Networking,High Te...",Ha Noi,Experienced (non-manager),"it system,antivirus,lan,wan,it support",5
842,1617065,chuyen vien ho tro nguoi dung hoi so it helpdesk,"IT - Software,IT - Hardware/Networking",Ha Noi,Experienced (non-manager),"it helpdesk,cong nghe thong tin,it phan cung,i...",6
3613,1624409,nhan vien ke toan junior accounting staff,Accounting,Ha Noi,Experienced (non-manager),"english,accounting,accountancy",7
2250,1622668,nhan vien ke toan sl 2 vi tri,"Accounting,Auditing,Finance/Investment","Vinh Phuc,Ha Noi",Experienced (non-manager),"tai chinh,kiem toan,ke toan tong hop,ke toan,a...",8
1523,1621688,it helpdesk staff accepted fresh,IT - Hardware/Networking,Ha Noi,Fresher/Entry level,"english,it helpdesk,lan,wan network",9


In [187]:
# u = '183.80.135.240, 10.122.10.10'
score_2, pred_2 = index2({"user_id": np.array([u])})
pre_items_2 = list(map(lambda x: x.decode('ascii'), pred_2.numpy()[0]))
check_df_2 = df_meta[df_meta._id.isin(list(map(lambda x: x.decode('ascii'), pred_2.numpy()[0])))][["_id","jobTitle", "industries", "locations", "jobLevel", "skills"]]
check_df_2["priority"] = check_df_2._id.apply(lambda x: pre_items_2.index(x))
check_df_2.sort_values(by=["priority"], ascending=True)

,_id,jobTitle,industries,locations,jobLevel,skills,priority
587,1614180,finance assistant manager pho phong tai chinh,"Accounting,Banking,Finance/Investment",Ho Chi Minh,Experienced (non-manager),"finance,finance analysis,accounting,management...",0
2322,1622751,financial analyst accounting staff supervisor,"Accounting,Auditing,Finance/Investment","Ho Chi Minh,Binh Duong",Experienced (non-manager),"finance analysis,accounting tax,account payable",1
2050,1622370,corporate banking rm,"Marketing,Banking,Finance/Investment",Ho Chi Minh,Experienced (non-manager),"english,corporate finance,bank credit,corporat...",2
3438,1624191,financial control and operations,"Insurance,Banking,Finance/Investment",Ho Chi Minh,Experienced (non-manager),"vas standards,ifrs,finance knowledgeable,finan...",3
2996,1623626,relationship manager large corporate,"Banking,Auditing,Finance/Investment",Ho Chi Minh,Manager,"relationship management,banking,finance",4
2843,1623436,cost control executive advanced in ap,"Accounting,Finance/Investment,Marine",Ho Chi Minh,Experienced (non-manager),"account payable,manage accounting process,engl...",5
3351,1624009,financial controller,"Accounting,Auditing,Finance/Investment",Ho Chi Minh,Manager,"finance controling,cpa,accounting,finance,kiem...",6
3567,1624291,finance supervisor,"Accounting,Auditing,Finance/Investment",Ho Chi Minh,Experienced (non-manager),"investment,finance,taxation,cost accounting,au...",7
2860,1623467,head of accounting finance department finance ...,"Accounting,Banking,Finance/Investment",Ho Chi Minh,Manager,"banking industry,accounting,management,financi...",8
3443,1624199,pfs business finance,"Insurance,Banking,Finance/Investment",Ho Chi Minh,Manager,"business finance,banking role,finance data ana...",9


In [184]:
# u = '183.80.135.240, 10.122.10.10'
score_1, pred_1 = index1({"user_id": np.array([u])})
pre_items_1 = list(map(lambda x: x.decode('ascii'), pred_1.numpy()[0]))
check_df_1 = df_meta[df_meta._id.isin(list(map(lambda x: x.decode('ascii'), pred_1.numpy()[0])))][["_id","jobTitle", "industries", "locations", "jobLevel", "skills"]]
check_df_1["priority"] = check_df_1._id.apply(lambda x: pre_items_1.index(x))
check_df_1.sort_values(by=["priority"], ascending=True)

,_id,jobTitle,industries,locations,jobLevel,skills,priority
587,1614180,finance assistant manager pho phong tai chinh,"Accounting,Banking,Finance/Investment",Ho Chi Minh,Experienced (non-manager),"finance,finance analysis,accounting,management...",0
2322,1622751,financial analyst accounting staff supervisor,"Accounting,Auditing,Finance/Investment","Ho Chi Minh,Binh Duong",Experienced (non-manager),"finance analysis,accounting tax,account payable",1
1030,1619510,finance specialist,"Accounting,Banking,Finance/Investment",Ho Chi Minh,Experienced (non-manager),"investment management,finance data analysis,pr...",2
3567,1624291,finance supervisor,"Accounting,Auditing,Finance/Investment",Ho Chi Minh,Experienced (non-manager),"investment,finance,taxation,cost accounting,au...",3
2552,1623051,assistant manager financial reporting tax,"Accounting,Administrative/Clerical,Finance/Inv...",Ho Chi Minh,Manager,"analyzing statistics,financial databases,finan...",4
3351,1624009,financial controller,"Accounting,Auditing,Finance/Investment",Ho Chi Minh,Manager,"finance controling,cpa,accounting,finance,kiem...",5
1164,1621224,chuyen vien ke toan gia thanh chi phi,"Food & Beverage,Auditing,Finance/Investment",Ho Chi Minh,Experienced (non-manager),"costing accounting,ke toan tong hop,accounting...",6
2420,1622868,commercial finance specialist,"Food & Beverage,Finance/Investment,FMCG",Ho Chi Minh,Experienced (non-manager),"finance data analysis,report analysis,financia...",7
1924,1622187,senior administration accountant ke toan kiem ...,"Accounting,Administrative/Clerical,Finance/Inv...","Dong Nai,Ho Chi Minh,Binh Duong",Experienced (non-manager),"accounting,ke toan hanh chinh,bao cao thue,eng...",8
2667,1623185,lead financial accountant manager level,"Accounting,Auditing,Finance/Investment",Ho Chi Minh,Manager,"computer skill,mathematics analysis,financial ...",9
